# Download the dataset & get the packeges

In [ ]:
!pip install transformers datasets
from IPython.display import clear_output
clear_output()

In [ ]:
!wget https://noisy-text.github.io/2017/files/wnut17train.conll
!wget https://noisy-text.github.io/2017/files/emerging.dev.conll
!wget https://noisy-text.github.io/2017/files/emerging.test.annotated
clear_output()

# Transofrm the data and convert to 🤗 Dataset

In [1]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            if len(line.split('\t'))  < 2:
                continue
            # print(line, len(line.split('\t')))
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)
    return token_docs, tag_docs



In [2]:
train_tokens, train_tags = read_wnut('wnut17train.conll')
val_tokens, val_tags = read_wnut('emerging.dev.conll')
test_tokens, test_tags = read_wnut('emerging.test.annotated')

In [3]:
total_tags = train_tags + val_tags + test_tags

unique_tags = set(tag for doc in total_tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

train_tag_id = [[tag2id[tag] for tag in doc] for doc in train_tags]
val_tag_id = [[tag2id[tag] for tag in doc] for doc in val_tags]
test_tag_id = [[tag2id[tag] for tag in doc] for doc in test_tags]

In [4]:
from datasets import  Dataset, DatasetDict

train_data = {
    'tokens': train_tokens, 
    'tag': train_tags,
    'tag_id': train_tag_id,
}

test_data = {
    'tokens': test_tokens,
    'tag': test_tags,
    'tag_id': test_tag_id,
}

validation_data = {
    'tokens': val_tokens,
    'tag': val_tags,
    'tag_id': val_tag_id,
}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(validation_data)
test_dataset = Dataset.from_dict(test_data)

whole_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

In [5]:
whole_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['tokens', 'tag', 'tag_id'],
        num_rows: 1287
    })
})

# Let the fun begin

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
tokenizer.is_fast

True

In [8]:
inputs = tokenizer(whole_dataset["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', '@', 'p', '##aul', '##walk', 'It', "'", 's', 'the', 'view', 'from', 'where', 'I', "'", 'm', 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'E', '##SB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '[SEP]']


In [9]:
print(inputs.word_ids())

[None, 0, 0, 0, 0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, None]


In [10]:
# labels in int
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [11]:
labels = whole_dataset["train"][0]["tag_id"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 2, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tag_id"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets = whole_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=whole_dataset["train"].column_names,
)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [16]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

2023-11-03 22:31:32.006908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 22:31:32.414308: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 22:31:32.414367: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 22:31:32.414434: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 22:31:32.582816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 22:31:32.584320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [17]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 35), dtype=int64, numpy=
array([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    5,    2,
           2,    0,    5,    6,    0,    0,    0,    0,    0,    0,    0,
           0, -100],
       [-100,    0,    0,    0,    0,    0,    0,    6,    6,    6,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        -100, -100]])>

In [18]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 2, 2, 0, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [19]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [20]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2tag,
    label2id=tag2id,
)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
